In [11]:
#!/usr/bin/python
from random import randint
import heapq
from datetime import datetime
import time
import pandas as pd
import numpy as np
import os
import threading

numOfWorkers=100
busyWorker=[False]*100   # for showing if the worker is busy working on something
flag=True
heapEntry=[] # a list of lists. Each entry of this list will be of the form -[priority, date time of creation, task name]
allTasksFinished=False
totalTime=0
#Every datapoint has a time associated with it using which we will schedule the tasks that are given.
#So we first read the order of incoming tasks(i.e. the type of task -PageRank, KMeans, SGD) to be picked alongwith their time of pickup from the file and then add all the flows of that particular task to the queue. 

intialTime=time.time()
print("intialTime",intialTime)

task_order_path='C:/Users/bodhv/Desktop/Sem2/Cloud/Project/Data/job_incoming_test.csv'
k_means_path='C:/Users/bodhv/Desktop/Sem2/Cloud/Project/Data/KMeans_test_jb_test.csv'
sgd_path='C:/Users/bodhv/Desktop/Sem2/Cloud/Project/Data/SGD_test_jb_test.csv'
page_rank_path='C:/Users/bodhv/Desktop/Sem2/Cloud/Project/Data/PageRank_test_jb_test.csv'

task_order_file=open(task_order_path, 'r')
k_means_file=open(k_means_path, 'r')
sgd_file=open(sgd_path, 'r')
page_rank_file=open(page_rank_path, 'r')
'''
df_task = pd.read_csv('C:/Users/bodhv/Desktop/Sem2/Cloud/Project/Data/job_incoming.txt', sep=",", names=['task_number', 'time_of_task'])
'''
df_task = pd.read_csv('C:/Users/bodhv/Desktop/Sem2/Cloud/Project/Data/job_incoming_test.csv', sep=",", names=['task_number', 'time_of_task'])
print(df_task)

df_k_means=pd.read_csv('C:/Users/bodhv/Desktop/Sem2/Cloud/Project/Data/KMeans_test_jb_test.csv', sep=",", names=['flow_size','job'])
df_sgd=pd.read_csv('C:/Users/bodhv/Desktop/Sem2/Cloud/Project/Data/SGD_test_jb_test.csv', sep=",", names=['flow_size','job'])
df_page_rank=pd.read_csv('C:/Users/bodhv/Desktop/Sem2/Cloud/Project/Data/PageRank_test_jb_test.csv', sep=",", names=['flow_size','job'])
df_k_means=df_k_means.tail(3)
df_sgd=df_sgd.tail(3)
df_page_rank=df_page_rank.tail(4)

'''
df_k_means=pd.read_csv('C:/Users/bodhv/Desktop/Sem2/Cloud/Project/Data/KMeans_test_jb.txt', sep=",", names=['time','previous_flow_time','agg_net_out','agg_net_in','agg_net_out_per_machine','agg_net_in_per_machine','tcp_queue','machine','cpu','ram','disk_write','memory_read','memory_write','first_call','flow_size','job'])
df_sgd=pd.read_csv('C:/Users/bodhv/Desktop/Sem2/Cloud/Project/Data/SGD_test_jb.csv', sep=",", names=['time','first_call','machine','agg_net_out_per_machine','agg_net_in_per_machine','agg_net_out','agg_net_in','flow_frequency','flow_size','job'])
df_page_rank=pd.read_csv('C:/Users/bodhv/Desktop/Sem2/Cloud/Project/Data/PageRank_test_jb.csv', sep=",", names=['time','flow_frequency','agg_net_out','agg_net_in','agg_net_out_per_machine','agg_net_in_per_machine','machine','flow_size','job'])
'''
print(df_k_means)
print(df_sgd)
print(df_page_rank)


intialTime 1587885353.3802488
   task_number  time_of_task
0            1          4000
1            2          1000
2            2          3000
3            3          3000
4            1           500
5            3          1000
  flow_size job
1     10000   1
2      5000   1
3     10000   2
  flow_size job
1      1000   1
2     10000   2
3      4000   2
  flow_size job
1     10000   1
2      1000   1
3      6000   2
4      6000   2


In [12]:
current_k_means_job=0
current_sgd_job=0
current_page_rank_job=0
current_task_index=-1
flagtime=False
print(len(df_task.index))
while current_task_index<len(df_task.index)-1: #repeats every second
#while current_task_index<3:
    allTasksFinished=False
    current_task_index=current_task_index+1
    #initial wait before you start running the first task
    if current_task_index==0:
        print("waiting for the first time")
        tempTime=float(df_task.time_of_task[current_task_index])/1000
        time.sleep(tempTime)
        
    print('current_task_index',current_task_index)
    cur_task_num=df_task.task_number[current_task_index]
    print("Inside first while\n")
    print('current task number',cur_task_num)
    if current_task_index==len(df_task.index)-1:
        curr_wait_time=0.0
    else:
        curr_wait_time=float(df_task.time_of_task[current_task_index+1])/1000 #time to wait before releasing new entry in secs
    #curr_wait_time=10.0
    start_wait_time=0.0
    starttime=time.time()
    print('1-curr_wait_time', curr_wait_time)
    print('1-start_wait_time', start_wait_time)
    
    
    

    if allTasksFinished==True:
        print("All tasks finished")
        break
        
    
    listOfTasks = {} 
    curr_df=None #represents the df which has info of only the current job of the given task

    # now read all the tasks that belong to that file(cur_task_num) that is chosen
    if cur_task_num==1:
        current_k_means_job=current_k_means_job+1
#        print('current_k_means_job',current_k_means_job)
        curr_df=df_k_means.loc[df_k_means['job'] == str(current_k_means_job)].copy()
#        print("k means job:", curr_df.job)
    elif cur_task_num==2:
        current_sgd_job=current_sgd_job+1
#        print('current_sgd_job', current_sgd_job)
        curr_df=df_sgd.loc[df_sgd['job'] == str(current_sgd_job)].copy()
#        print("sgd job:", curr_df.job)
    elif cur_task_num==3:
        current_page_rank_job=current_page_rank_job+1
#        print('current_page_rank_job',current_page_rank_job)
        curr_df=df_page_rank.loc[df_page_rank['job'] == str(current_page_rank_job)].copy()
#        print('page rank job',curr_df.job)

    #print(curr_df)
    #curr_df=curr_df.head(5)
    curr_df=pd.DataFrame(curr_df)
    numOfNewTasks=len(curr_df.index)
    currLenDict=len(listOfTasks)
    #when new entries enter the queue:
#    print("number of new tasks:", numOfNewTasks)
    #print("initial listOfTasks", listOfTasks)
    
    print("curr_df:",curr_df)
    start_index=curr_df.head(1).index[0]
    end_index=curr_df.tail(1).index[0]
    print('start index', start_index)
    print('end index', end_index)
    #for x in range(currLenDict+1,currLenDict+numOfNewTasks+1):
    for x in range(start_index,numOfNewTasks+start_index):
        key="Task"+str(cur_task_num)+"Job"+str(start_index)
        val=1 #priority at entry time
        now = datetime.now() # time object -- now.strftime("%Y%m%d%H%M%S") 
        print('x',x)
        #print('curr_df',curr_df)
        timeToCompletion=float(curr_df.flow_size[x])/1000
        #print('time to completion', timeToCompletion)

        listOfTasks[key]=[val,now.strftime("%Y%m%d%H%M%S"),timeToCompletion]
#        print('listOfTaks['+str(key)+']='+str(listOfTasks[key]))

    #print(" final listOfTasks",listOfTasks)

    #create a heap for the first time:
    if flag==True:
        print("creating heap for the first time")
        for keyTaskName in listOfTasks:
            tempList=[]
            tempList.append(listOfTasks[keyTaskName][0])      #priority
            tempList.append(int(listOfTasks[keyTaskName][1])) #time of creation
            tempList.append(listOfTasks[keyTaskName][2])      #time required to complete this task
            tempList.append(keyTaskName)                      #task name
#            print('tempList',tempList)
            heapEntry.append(tempList)
#        print("list is:", heapEntry)
        heapq.heapify(heapEntry) 
#        print("created heap for the first time")
#        print("heap is:", heapEntry)
        flag=False
    else:
#        print(" adding elements to heap")
        for keyTaskName in listOfTasks:
            tempList=[]
            tempList.append(listOfTasks[keyTaskName][0])      #priority
            tempList.append(int(listOfTasks[keyTaskName][1])) #time of creation
            tempList.append(listOfTasks[keyTaskName][2])      #time required to complete this task
            tempList.append(keyTaskName)                      #task name
            heapq.heappush(heapEntry,tempList)
#            print('element added to heap:', tempList)
#            print("current heap:", heapEntry)

    while True:
#        print("inside 2nd while")
        if len(heapEntry)>0: # while the heap is not empty i.e. there are tasks yet to finish
            if start_wait_time>curr_wait_time:
                break
            
#            print("heap length:", len(heapEntry))
            if all(boolean==True for boolean in busyWorker):
#                print("All workers busy")
                continue

            for x in range(numOfWorkers):
                if busyWorker[x]==False and len(heapEntry)>0:
#                    print("worker",x,"is free")
                    busyWorker[x]=True
                    tempTask=heapq.heappop(heapEntry)
#                    print("popped entry:", tempTask)
                    tempTask[2]=tempTask[2]-1 #reduced remaining completion time by 1 sec
                    if tempTask[2]<0:
#                        print("temptask finished:", tempTask)
                        continue # will not add that task in the heap again as it is finished
                    else:

                        tempTask[0]=tempTask[0]+1
#                        print("temptask remaining work:", tempTask)
                        heapq.heappush(heapEntry, tempTask)
#                        print("pushed", tempTask, "back into heap")
#                        print("new heap:", heapEntry)
            print("going to sleep for a sec")
#            print("time.time()%1.0", time.time()%1.0)
#            print("starttime%1.0", starttime%1.0)
#            print("sleeping for", 1.0-(time.time()-starttime)% 1.0)
            time.sleep(1.0-(time.time()-starttime)%1.0) #wait for one second
            if flagtime==False:
                start_wait_time=float(str(time.time()))
                flagtime=True
            start_wait_time=start_wait_time+1
#            print("going out of sleep")
            for x in range(numOfWorkers): #release all workers from work after every second
                busyWorker[x]=False
#                print("released all workers from work")
            #Add break inner while loop here to get new entries in list every second.
            #break

        else:
            print("mark all tasks finished tag as true")
            allTasksFinished=True
            break
    if start_wait_time<curr_wait_time:
        print('2-curr_wait_time', curr_wait_time)
        print('2-start_wait_time', start_wait_time)
        print("Completed task, now sleeping for", curr_wait_time-(time.time()-starttime)%curr_wait_time)
        time.sleep(curr_wait_time-(time.time()-starttime)%curr_wait_time) #wait for one second
#        print("Going out of sleep between adding new entries to ")
totalTime=time.time()-intialTime
print("Total time:",totalTime)
                                        

6
waiting for the first time
current_task_index 0
Inside first while

current task number 1
1-curr_wait_time 10.0
1-start_wait_time 0.0
curr_df:   flow_size job
1     10000   1
2      5000   1
start index 1
end index 2
x 1
x 2
creating heap for the first time
going to sleep for a sec
mark all tasks finished tag as true
current_task_index 1
Inside first while

current task number 2
1-curr_wait_time 30.0
1-start_wait_time 0.0
curr_df:   flow_size job
1      1000   1
start index 1
end index 1
x 1
going to sleep for a sec
mark all tasks finished tag as true
2-curr_wait_time 30.0
2-start_wait_time 1.0
Completed task, now sleeping for 28.98047685623169
current_task_index 2
Inside first while

current task number 2
1-curr_wait_time 30.0
1-start_wait_time 0.0
curr_df:   flow_size job
2     10000   2
3      4000   2
start index 2
end index 3
x 2
x 3
going to sleep for a sec
mark all tasks finished tag as true
2-curr_wait_time 30.0
2-start_wait_time 1.0
Completed task, now sleeping for 28.979914

In [55]:
# threading module


exitFlag = 0

class myThread (threading.Thread):
   def __init__(self, threadID, name, counter):
      threading.Thread.__init__(self)
      self.threadID = threadID
      self.name = name
      self.counter = counter
   def run(self):
      print "Starting " + self.name
      print_time(self.name, 5, self.counter)
      print "Exiting " + self.name

def print_time(threadName, counter, delay):
   while counter:
      if exitFlag:
         threadName.exit()
      time.sleep(delay)
      print "%s: %s" % (threadName, time.ctime(time.time()))
      counter -= 1

# Create new threads
thread1 = myThread(1, "Thread-1", 1)
thread2 = myThread(2, "Thread-2", 2)

# Start new Threads
thread1.start()
thread2.start()

print "Exiting Main Thread"

SyntaxError: Missing parentheses in call to 'print'. Did you mean print("Starting " + self.name)? (<ipython-input-55-803c7ba956a3>, line 13)

In [15]:

while True: #repeats every second
    current_task_index=current_task_index+1
    cur_task_num=df_task.task_number[current_task_index]
    print("Inside first while")
    starttime=time.time()
    if allTasksFinished==True:
        totalTime=datetime.now()-initialStartTime
        print("Total time:", totalTime.strftime("%H-%M-%S"))
        print("All tasks finished")
        break

    listOfTasks = {} 
    # now read all the tasks that belong to that file that is chosen

    numOfNewTasks=randint(1,5)
    currLenDict=len(listOfTasks)
    # when new entries enter the queue:
    print("number of new tasks:", numOfNewTasks)
    print(" initial listOfTasks",listOfTasks)

    for x in range(currLenDict+1,currLenDict+numOfNewTasks+1):

        key="Task"+str(x)
        val=1
        now = datetime.now() # time object -- now.strftime("%Y%m%d%H%M%S") 
        timeToCompletion=randint(1,10)
        listOfTasks[key]=[val,now.strftime("%Y%m%d%H%M%S"),timeToCompletion]

    print(" final listOfTasks",listOfTasks)

    #create a heap for the first time:
    if flag==True:
        print(" creating heap for the fisrt time")
        for keyTaskName in listOfTasks:
            tempList=[]
            tempList.append(listOfTasks[keyTaskName][0])      #priority
            tempList.append(int(listOfTasks[keyTaskName][1])) #time of creation
            tempList.append(listOfTasks[keyTaskName][2])	  #time required to complete this task
            tempList.append(keyTaskName)                      #task name
            heapEntry.append(tempList)
        heapq.heapify(heapEntry) 
        print(" created heap for the fisrt time")
        print("heap is:", heapEntry)
        flag=False
    else:
        print(" adding elements to heap")
        for keyTaskName in listOfTasks:
            tempList=[]
            tempList.append(listOfTasks[keyTaskName][0])      #priority
            tempList.append(int(listOfTasks[keyTaskName][1])) #time of creation
            tempList.append(listOfTasks[keyTaskName][2])	  #time required to complete this task
            tempList.append(keyTaskName)                      #task name
            heapq.heappush(heapEntry,tempList)
            print('element added to heap:', tempList)
            print("current heap:", heapEntry)

    while True:
        print("inside 2nd while")
        if len(heapEntry)>0:
            print("heap length:", len(heapEntry))
            if all(boolean==True for boolean in busyWorker):
                print("All workers busy")
                continue

            for x in range(numOfWorkers):
                if busyWorker[x]==False and len(heapEntry)>0:
                    print("worker",x,"is free")
                    busyWorker[x]=True
                    tempTask=heapq.heappop(heapEntry)
                    print("popped entry:", tempTask)
                    tempTask[2]=tempTask[2]-1 #reduced remaining completion time by 1 sec
                    if tempTask[2]==0:
                        print("temptask finished:", tempTask)
                        continue # will not add that task in the heap again as it is finished
                    else:

                        tempTask[0]=tempTask[0]+1
                        print("temptask remaining work:", tempTask)
                        heapq.heappush(heapEntry, tempTask)
                        print("pushed", tempTask, "back into heap")
                        print("new heap:", heapEntry)
            print("going to sleep for a sec")
            print("time.time()%1.0", time.time()%1.0)
            print("starttime%1.0", starttime%1.0)
            print("sleeping for", 1.0-(time.time()-starttime)% 1.0)
            time.sleep(1.0-(time.time()-starttime)%1.0) #wait for one second
            print("going out of sleep")
            for x in range(numOfWorkers):	#release all workers from work after every second
                busyWorker[x]=False
                print("released all workers from work")
            #Add break inner while loop here to get new entries in list every second.
            #break;	

        else:
            print("mark all tasks finished tag as true")
            allTasksFinished=True
            break



SyntaxError: invalid syntax (<ipython-input-15-ff47c9c660ad>, line 1)